#### Dimensionality
From what we've learned so far, how can we calculate the number of neurons of each layer in our CNN?

Given:

- our input layer has a width of W and a height of H
- our convolutional layer has a filter size F
- we have a stride of S
- a padding of P
- and the number of filters K,

the following formula gives us the width of the next layer: $W_out = (W−F+2P)/S+1$.

The output height would be: $H_out = (H-F+2P)/S + 1$.

And the output depth would be equal to the number of filters $D_out = K$.

The output volume would be $W_out * H_out * D_out$.

In [66]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [67]:
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)
print(mnist.train.images[0].shape)
print(mnist.train.labels[0].shape)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
(28, 28, 1)
(10,)


In [68]:
## clear graph
tf.reset_default_graph()

In [69]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128
# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256
n_classes = 10  # MNIST total classes (0-9 digits)
keep_rate = 0.75  # Dropout, probability to keep units

In [70]:
## set up weights 
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),      ## first converlution layer, 5x5x1 window, 32 layers
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),     ## from 32 to 64 layers 
    'wd1': tf.Variable(tf.random_normal([7*7*64,1024])),   ## a fully connected layer, turn everything into 1024 dimentions
    'out': tf.Variable(tf.random_normal([1024,n_classes])) ## output layer
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [71]:
## set up conv function 
def conv2d(x,W,b,s=1):
    x = tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='SAME')
    x = tf.nn.bias_add(x,b)     ## tf.add() doesn't work when the tensors aren't the same shape.
    return tf.nn.relu(x)        ## use relu as activation function 

## set up Max  Pooling layer
def maxpool2d(x,k=2):
    return tf.nn.max_pool(
        x,
        ksize = [1,k,k,1],      ## k is the size of the filter 
        strides=[1,k,k,1],
        padding='SAME'
    )

In [72]:
## build the conv network 
def conv_net(x,weights,biases,keep_rate):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x,weights['wc1'],biases['bc1'])   ## stride defaults to be 1, we are not reducing pic size here
    conv1 = maxpool2d(conv1,k=2)                     ## reduce the picture by 2 
    
    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'],biases['bc2'])
    conv2 = maxpool2d(conv2,k=2)
    
    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])  ## first reshape image into stacked row vectors
    fc1 = tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    tf1= tf.nn.dropout(fc1,keep_rate)
    
    # Output Layer - class prediction - 1024 to 10 
    out = tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    return out 


In [ ]:
##########
# Set up inputes, cost, and optimizer 
##########

# graph input
x = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables 
init = tf.global_variables_initializer()


In [ ]:
##############
## Run session 
##############

with tf.Session() as sess:
    sess.run(init)  ## initialize variables 
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer,feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: keep_rate})
            
        ##########################################
        ## print out resulsts after every 20 batches
        #########################################
             if batch % 20 == 0:
                # Calculate batch loss and accuracy
                loss = sess.run(cost, feed_dict={
                    x: batch_x,
                    y: batch_y,
                    keep_prob: 1.})
                valid_acc = sess.run(accuracy, feed_dict={
                    x: mnist.validation.images[:test_valid_size],
                    y: mnist.validation.labels[:test_valid_size],
                    keep_prob: 1.})


                print('Epoch {:>2}, Batch {:>3} -'
                      'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                    epoch + 1,
                    batch + 1,
                    loss,
                    valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 65012.9648 Validation Accuracy: 0.117188
Epoch  1, Batch   2 -Loss: 55055.8594 Validation Accuracy: 0.101562
Epoch  1, Batch   3 -Loss: 39513.8984 Validation Accuracy: 0.058594
Epoch  1, Batch   4 -Loss: 33135.2969 Validation Accuracy: 0.117188
Epoch  1, Batch   5 -Loss: 29975.9004 Validation Accuracy: 0.093750
Epoch  1, Batch   6 -Loss: 30617.9609 Validation Accuracy: 0.117188
Epoch  1, Batch   7 -Loss: 29096.4453 Validation Accuracy: 0.128906
Epoch  1, Batch   8 -Loss: 25309.4180 Validation Accuracy: 0.128906
Epoch  1, Batch   9 -Loss: 22404.5859 Validation Accuracy: 0.121094
Epoch  1, Batch  10 -Loss: 20597.3223 Validation Accuracy: 0.121094
Epoch  1, Batch  11 -Loss: 22713.8223 Validation Accuracy: 0.140625
Epoch  1, Batch  12 -Loss: 18416.7812 Validation Accuracy: 0.144531
Epoch  1, Batch  13 -Loss: 19094.1992 Validation Accuracy: 0.148438
Epoch  1, Batch  14 -Loss: 18684.6445 Validation Accuracy: 0.160156
Epoch  1, Batch  15 -Loss: 19244.3633 Validation